## Case Study: Cyclistic, a bike sharing company

### Preface: Cyclistic is a fictional company located in Chicago, Illinois. This company features more than 5,800 bicycles and 600 docking stations.
### Cyclistic’s finance analysts have concluded that annual members are much more profitable than casual riders. Although the pricing flexibility helps Cyclistic attract more customers, the director of marketing believes that maximizing the number of annual members will be key to future growth.

### Goal: : Design marketing strategies aimed at converting casual riders into annual members.

#### Focal questions :
#### 1. How do annual members and casual riders use Cyclistic bikes differently?
#### 2. Why would casual riders buy Cyclistic annual memberships?
#### 3. How can Cyclistic use digital media to influence casual riders to become members?

#### The director of marketing has assigned me with the first question to answer: How do annual members and casual riders use Cyclistic bikes
#### differently?
#### Produce a report with the following deliverables:
#### 1. A clear statement of the business task
#### 2. A description of all data sources used
#### 3. Documentation of any cleaning or manipulation of data
#### 4. A summary of your analysis
#### 5. Supporting visualizations and key findings
#### 6. Your top three recommendations based on your analysis

In [442]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from matplotlib import pyplot as plt
pd.options.mode.chained_assignment = None

In [443]:
file1 = pd.read_csv("/kaggle/input/divvytrips-data/Divvy_Trips_2020_Q1.csv")
file2 = pd.read_csv("/kaggle/input/divvytrips-data/Divvy_Trips_2019_Q2.csv")
file3 = pd.read_csv("/kaggle/input/divvytrips-data/Divvy_Trips_2019_Q3.csv")
file4 = pd.read_csv("/kaggle/input/divvytrips-data/Divvy_Trips_2019_Q4.csv")
pd.set_option('display.width', 512)
pd.set_option('display.max_columns', 20)

## Align datasets before combining

In [444]:
file1 = file1.drop(["ride_id", "rideable_type", "start_lat", "start_lng", "end_lat", "end_lng", "start_station_id", "end_station_id"], axis = 1)
file1 = file1[["started_at", "ended_at", "start_station_name", "end_station_name", "member_casual"]]
# Remove unwanted columns, and rename kept columns
file2 = file2.drop(["01 - Rental Details Rental ID", "01 - Rental Details Bike ID", "03 - Rental Start Station ID", "01 - Rental Details Duration In Seconds Uncapped", "02 - Rental End Station ID", "Member Gender", "05 - Member Details Member Birthday Year"], axis = 1)

file2.columns = ["started_at", "ended_at",
"start_station_name", "end_station_name", "member_casual"]

# Alter file3
file3 = file3.drop(["trip_id", "bikeid", "from_station_id", "tripduration", "to_station_id", "gender", "birthyear"], axis = 1)
file3.columns = ["started_at", "ended_at",
"start_station_name", "end_station_name", "member_casual"]

# Alter file4
file4 = file4.drop(["trip_id", "bikeid", "tripduration", "from_station_id", "to_station_id", "gender", "birthyear"], axis = 1)
file4.columns = ["started_at", "ended_at",
"start_station_name", "end_station_name", "member_casual"]
#print(file2.start_station_name.value_counts())
# concat dataframes
total = [file1, file2, file3, file4]
df = pd.concat(total, axis = 0)

#### The business task is to analyze data on riders, specifically the different behaviors of members vs casual customers.

#### In order to complete this task, I will be using the provided datasets which were licensed and made available by
#### Motivate International Inc. The license can be found here: [link]https://www.divvybikes.com/data-license-agreement. 

### Reduce unneccesary values for simplication

In [445]:
df.loc[df.member_casual == 'Subscriber', 'member_casual'] = 'member'
df.loc[df.member_casual == 'Customer', 'member_casual'] = 'casual'


### Convert numerical info to date info

In [446]:
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])
print(df.head())

### Make dataset with columns designed to focus on trip duration and travel habits by casuals/members

In [447]:
df_time = df.copy()
df_time['trip_duration'] = df_time['ended_at'] - df_time['started_at']
df_time['trip_duration_minutes'] = df_time['trip_duration'].dt.seconds.div(60).astype('int')
df_time = df_time.drop(['start_station_name', 'end_station_name', 'trip_duration'], axis = 1)
df_time = df_time.sort_values(['trip_duration_minutes'])
#remove rides that were less than a minute long
df_time.drop(df_time.index[df_time['trip_duration_minutes'] == 0], inplace = True)
print(df_time.head())


In [448]:
df_time['duration_groups'] = pd.qcut(df_time['trip_duration_minutes'],
                                     5)
df_time['tdm_labels'] = pd.qcut(df_time['trip_duration_minutes'],
                                5,
                                labels = ['Six Minutes or Less', 'Six to Nine Minutes', 'Nine to Fourteen Minutes',
                                          'Fourteen to Twenty Four Minutes', 'Over 24 Minutes'])

df_time = df_time[['member_casual', 'trip_duration_minutes', 'duration_groups', 'tdm_labels']]

In [449]:
slotted_durations = (df_time[['member_casual', 'tdm_labels', 'duration_groups']]
                     .groupby(['tdm_labels', 'duration_groups'])
                     .count()
                     .reset_index()
                     .rename(columns = {'tdm_labels': 'labels',
                                         'member_casual':'member_casual_count'})
                     .dropna()
                     .reset_index(drop = True))
slotted_durations['member_casual_count'] = slotted_durations['member_casual_count'].astype(int)

In [450]:
for limit, threshold in [('upper', 1),
                         ('lower', 0)]:
    slotted_durations[limit] = (slotted_durations['duration_groups'].astype(str)
                               .str.split(expand=True)[threshold]
                               .str.strip('(,] ')
                               .astype(float))
slotted_durations['bin_size'] = (slotted_durations['upper'] - slotted_durations['lower']).round()
slotted_durations[['labels', 'duration_groups', 'bin_size', 'member_casual_count']]

### Creat a pivot table showing segmented trip duration between rider types

In [451]:
p = (pd.pivot_table(df_time[['member_casual','tdm_labels']],
                    index='member_casual',
                    columns='tdm_labels',
                    aggfunc=len)
     .fillna(0)
     .astype(int))
p.columns = p.columns.astype(str)
p = p.reset_index()
p.columns.name = ''
p.columns = p.columns.str.title()
print(p)

### Create stacked column chart showing data

In [452]:
%matplotlib inline

import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')

def create_stacked_bar_plot(p,
                            y_label,
                            title):
    plt.figure(figsize=(8,6))

    rows = len(p.index)

    # List of zeros with length equal to the number of rows in the pivot table
    bottoms = [0] * rows

    # Create the stacks
    for col in p.columns[1:]:
        plt.bar(p[p.columns[0]],
                p[col],
                label=col,
                width=0.5,
                bottom=bottoms)
        bottoms += p[col].values

    # Add denominators at the top of each bar
    heights = list(bottoms) # bottoms now contains the height of each column overall
    xs = range(0,len(p.index),1)
    
    # Make plot slightly taller than the max height
    plt.gca().set_ylim(0, max(heights)*1.1)

    for x, height in zip(xs, heights):
        plt.gca().text(x,
                       height + max(heights)*.01,
                       str(height),
                       ha='center',
                       va='bottom',
                       color='black',
                       fontsize=16)

    # Add percentages to each stack
    rects = plt.gca().patches

    for n, r in enumerate(rects):
        height = r.get_height()

        if height >= max(heights)*0.1: # If the height of the rectangle is large enough to be labeled
            perc = '{0:.0f}%'.format(height/heights[n % rows]*100)
            perc_w_denom = '{0:.0f}%'.format((height/heights[n % rows])*100) + '\n(' + str(height) + ')'

            plt.gca().text(r.get_x() + r.get_width() / 2,
                           r.get_y() + height / 2,
                           perc_w_denom,
                           ha='center',
                           va='center',
                           color='white',
                           fontsize=16)

    # Hide labels and ticks on left to improve data-ink ratio
    plt.tick_params(
    axis='y',
    left=False,
    labelleft=False)

    # Hide frame to improve data-ink ratio
    for spine in plt.gca().spines.values():
        spine.set_visible(False)

    # Add legend at the Top
    plt.gca().legend(loc='lower center',
                     borderpad = 2,
                     bbox_to_anchor=(0.45, 1),
                     ncol=4,
                     frameon=False,
                     prop={'size':20})

    plt.ylabel(y_label)
    plt.title(title);

In [453]:
create_stacked_bar_plot(p,
                        'Rental Time',
                        "Trip Duration - Overall and segmented)")

### This visualization shows that casual riders are more interested in longer rides, while over half of members ride for less than ten minutes

### Next step will be looking into whether casuals and members return bikes at same stations or different stations

In [ ]:
df_stations = df[['start_station_name', 'end_station_name', 'member_casual']].copy()
print(df_stations.head())
df_stations['start_station_name'] = df_stations['start_station_name'].astype(str)
df_stations['end_station_name'] = df_stations['end_station_name'].astype(str)
print(df_stations.dtypes)
# df['count'] = df.apply(lambda x: 1 if x.category in x.category_pred else 0, axis = 1)
df_stations['same_station'] = df_stations.apply(lambda x: 1 if x.start_station_name in x.end_station_name else 0, axis = 1)
print(df_stations.head())
print(df_stations.groupby(['member_casual', 'same_station'].value_counts()))

### The following will delve into rider types and their weekend/weekday biases

#### Create filtered dataframes for weekdays/weekends

In [ ]:
pd.options.display.float_format = '{:,.0f}'.format
df2 = df
df2['days'] = df2['started_at'].dt.day_name()
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
weekends = ['Saturday', 'Sunday']
weekday_df = df2[df2['days'].isin(weekdays)]
weekend_df = df2[df2['days'].isin(weekends)]
print(weekday_df.head())
print(weekend_df.head())


### Using pie charts, show discrepency between casual and member riders, as well as weekday/weekend riders

In [ ]:
x = df2['member_casual'].value_counts()
mylabels = ['Members', 'Casuals']
plt.style.use('_mpl-gallery-nogrid')
plt.pie(x, labels = mylabels, autopct='%1.1f%%')
plt.title("Overall Members vs Casuals")
plt.axis('equal')
plt.show()

x = weekday_df.loc[weekday_df.member_casual == 'member', 'member_casual'].count()
y = weekend_df.loc[weekend_df.member_casual == 'member', 'member_casual'].count()
z = weekday_df.loc[weekday_df.member_casual == 'casual', 'member_casual'].count()
z1 = weekend_df.loc[weekend_df.member_casual == 'casual', 'member_casual'].count()
mylabels = ['M-F Members', 'Weekend Members', 'M-F Casuals', 'Weekend Casuals']
plt.style.use('_mpl-gallery-nogrid')
plt.pie([x,y,z,z1], labels = mylabels, autopct='%1.1f%%')
plt.title("Members and Casuals broken down by work week/weekends")
plt.axis('equal')
plt.show()


## Based on this data, my belief is the best group of casuals to convert into members would be M-F Casuals, as the majority of members use the service during those days. That group currently makes up 13.3% of overall users

### Additionally, I think insight into which stations are used the most and by whom can be advantageous.

####  Create dataframes to gain insight on which destinations are most popular and by whom, starting with destinations

In [ ]:
df3 = df2.groupby(['end_station_name'])
mem_end = df3['member_casual'].apply(lambda x: x.str.contains('member').sum())
mem_end_sorted = pd.DataFrame(mem_end.sort_values(ascending=False))
mem_end_sorted = mem_end_sorted.rename(columns={'member_casual':'members'})

cas_end = df3['member_casual'].apply(lambda x: x.str.contains('casual').sum())
cas_end_sorted = pd.DataFrame(cas_end.sort_values(ascending=False))
cas_end_sorted = cas_end_sorted.rename(columns = {'member_casual':'casuals'})

end_stations_sorted = pd.concat([mem_end_sorted, cas_end_sorted], axis = 'columns')
end_stations_sorted['Total'] = mem_end_sorted['members'] + cas_end_sorted['casuals']
df4 = end_stations_sorted.sort_values(['Total'], ascending = False)
df4 = df4.reset_index()
print(df4.head(10))

#### Graphs showing top 25 destinations, and who is using them (members/casuals)

In [ ]:
df5 = df4.head(25)
x = df5['end_station_name']
y = df5['members']
z = df5['casuals']
t = df5['Total']
plt.figure(figsize = (10,10))
plt.barh(x, t)
plt.gca().invert_yaxis()
plt.title("Our most popular destinations")
plt.xlabel("Number of passengers")
plt.ylabel("Destinations")
plt.show()
plt.savefig('destinations.png')

plt.figure(figsize = (10,10))
plt.barh(x, y)
plt.gca().invert_yaxis()
plt.title("Members use at our most popular destinations")
plt.xlabel("Number of passengers")
plt.ylabel("Destinations")
plt.show()
plt.savefig('member_dest.png')

plt.figure(figsize = (10,10))
plt.barh(x, z)
plt.gca().invert_yaxis()
plt.title("Casuals use at our most popular destinations")
plt.xlabel("Number of passengers")
plt.ylabel("Destinations")
plt.show()
plt.savefig('casual_dest.png')


## As previously stated, it is my belief that the casual riders that will be best suited by purchasing a subcription and becoming a member are the casual riders that mimic members riding behavior the most. With that said, I will be trying to isolate casual riders that use this service during the week at stations that are heavily used by members as opposed to stations regularly frequented by casual riders.

## The final step will be showing start_stations and end_stations that are heavily used by members during the work week.

#### Creating needed dataframe

In [ ]:
pd.set_option('display.max_columns', 50)
df4 = df4.drop(['Total'], axis = 1)
df4.iloc[:, 1:] = df4.iloc[:, 1:].div(df4.iloc[:, 1:].sum(1), axis=0).mul(100)
filt = df4['members'] >= 90
print(df4[filt].head(25))


In [ ]:
df5 = df2.groupby(['start_station_name'])
mem_start = df5['member_casual'].apply(lambda x: x.str.contains('member').sum())
mem_start_sorted = pd.DataFrame(mem_start.sort_values(ascending=False))
mem_start_sorted = mem_start_sorted.rename(columns={'member_casual':'members'})

cas_start = df5['member_casual'].apply(lambda x: x.str.contains('casual').sum())
cas_start_sorted = pd.DataFrame(cas_start.sort_values(ascending=False))
cas_start_sorted = cas_start_sorted.rename(columns = {'member_casual':'casuals'})

start_stations_sorted = pd.concat([mem_start_sorted, cas_start_sorted], axis = 'columns')
start_stations_sorted['Total'] = mem_start_sorted['members'] + cas_start_sorted['casuals']
df6 = start_stations_sorted.sort_values(['Total'], ascending = False)
df6 = df6.reset_index()
df6 = df6.drop(['Total'], axis = 1)
df6.iloc[:, 1:] = df6.iloc[:, 1:].div(df6.iloc[:, 1:].sum(1), axis=0).mul(100)

filt = df6['members'] >= 90
print(df6[filt].head(25))

## these lists show, in order of overall traffic, the stations where casual riders are most likely to be interested in becoming members